Import modules

In [1]:
import myosuite
import gym
import skvideo.io
import numpy as np
import os
import imageio
import tqdm
import time

MyoSuite:> Registering Myo Envs


Display Settings

In [2]:
from IPython.display import HTML
from base64 import b64encode

def show_video(video_path, video_width = 400):

  video_file = open(video_path, "r+b").read()

  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"""<video autoplay width={video_width} controls><source src="{video_url}"></video>""")


<h2> Load Policy </h2>
<p>This code specifically uses unpickling to get the policy (mjrl). </p>

In [3]:
env = gym.make('CenterReachOut-v0')
policy = "iterations/policy_2900.pickle" #Change the folder to the policy location

import pickle
# load policy
pi = pickle.load(open(policy, 'rb'))

all_rewards = []

frames = []
for _ in range(20): # 20 random targets
  env.reset()
  ep_rewards = []
  done = False
  obs = env.reset()
  while not done:
      o = env.get_obs()
      # get the next action from the policy
      action, _ = pi.get_action(o)
      # take an action based on the current observation
      obs, reward, done, info = env.step(action)
      ep_rewards.append(reward)
  all_rewards.append(np.sum(ep_rewards))

print(f"Average reward: {np.mean(all_rewards)} over 20 episodes")


env.close()


    MyoSuite: A contact-rich simulation suite for musculoskeletal motor control
        Vittorio Caggiano, Huawei Wang, Guillaume Durandau, Massimo Sartori, Vikash Kumar
        L4DC-2019 | https://sites.google.com/view/myosuite
    


IndexError: index 1 is out of bounds for axis 0 with size 1

### Camera Id's:
// 0 - Side facing
// 1 - Front facing
// 2 - Above facing

In [ ]:
# Use sleep if you want to examine certain steps during the episodes
from time import sleep

# Render trained policy
frames = []
for _ in range(50): # 5 random targets
  env.reset()
  ep_rewards = []
  done = False
  obs = env.reset()
  for _ in range(15):
      print(env.obj_xyz_range)
      frame = env.sim.renderer.render_offscreen(
                        width=400,
                        height=400,
                        camera_id=1)
      frames.append(frame)
      o = env.get_obs()
      # get the next action from the policy
      action, _ = pi.get_action(o)
      # take an action based on the current observation
      obs, reward, done, info = env.step(action)

env.close()

os.makedirs('videos', exist_ok=True)
video_path = 'videos/test_policy3.mp4'
# make a local copy
imageio.mimsave(video_path, frames, fps=30)
show_video('videos/test_policy3.mp4')

In [ ]:
import collections
import gym
import numpy as np
from myosuite.envs.myo.base_v0 import BaseV0

class ReachEnvV0(BaseV0):
    DEFAULT_OBS_KEYS = ['hand_qpos', 'hand_qvel', 'obj_pos', 'reach_err']
    DEFAULT_RWD_KEYS_AND_WEIGHTS = {
        "reach": 1.0,
        "bonus": 4.0,
        "penalty": 50,
    }

    def __init__(self, model_path, obsd_model_path=None, seed=None, **kwargs):

        # EzPickle.__init__(**locals()) is capturing the input dictionary of the init method of this class.
        # In order to successfully capture all arguments we need to call gym.utils.EzPickle.__init__(**locals())
        # at the leaf level, when we do inheritance like we do here.
        # kwargs is needed at the top level to account for injection of __class__ keyword.
        # Also see: https://github.com/openai/gym/pull/1497
        gym.utils.EzPickle.__init__(self, model_path, obsd_model_path, seed, **kwargs)

        # This two step construction is required for pickling to work correctly. All arguments to all __init__
        # calls must be pickle friendly. Things like sim / sim_obsd are NOT pickle friendly. Therefore we
        # first construct the inheritance chain, which is just __init__ calls all the way down, with env_base
        # creating the sim / sim_obsd instances. Next we run through "setup"  which relies on sim / sim_obsd
        # created in __init__ to complete the setup.
        super().__init__(model_path=model_path, obsd_model_path=obsd_model_path, seed=seed, env_credits=self.MYO_CREDIT)

        self._setup(**kwargs)

    def _setup(self,
               obj_xyz_range=None,
               far_th=.35,
               obs_keys: list = DEFAULT_OBS_KEYS,
               drop_th=0.50,
               qpos_noise_range=None,
               weighted_reward_keys: dict = DEFAULT_RWD_KEYS_AND_WEIGHTS,
               **kwargs,
               ):
        self.far_th = far_th
        self.palm_sid = self.sim.model.site_name2id("handsite")
        self.object_sid = self.sim.model.site_name2id("object_o")
        self.object_bid = self.sim.model.body_name2id("Object")
        self.obj_xyz_range = obj_xyz_range
        self.drop_th = drop_th
        self.qpos_noise_range = qpos_noise_range
        super()._setup(obs_keys=obs_keys,
                       weighted_reward_keys=weighted_reward_keys,
                       **kwargs,
                       )
        keyFrame_id = 0 if self.obj_xyz_range is None else 1
        self.init_qpos[:] = self.sim.model.key_qpos[keyFrame_id].copy() #Initial position for the hand

    def get_obs_vec(self):
        self.obs_dict['time'] = np.array([self.sim.data.time])
        self.obs_dict['hand_qpos'] = self.sim.data.qpos[:].copy()
        self.obs_dict['hand_qvel'] = self.sim.data.qvel[:].copy() * self.dt
        if self.sim.model.na > 0:
            self.obs_dict['act'] = self.sim.data.act[:].copy()

        # reach error
        obs_dict['obj_pos'] = sim.data.site_xpos[self.object_sid]
        obs_dict['palm_pos'] = sim.data.site_xpos[self.palm_sid]
        self.obs_dict['reach_err'] = np.array(self.obs_dict['palm_pos']) - np.array(self.obs_dict['obj_pos'])

        t, obs = self.obsdict2obsvec(self.obs_dict, self.obs_keys)
        return obs

    def get_obs_dict(self, sim):
        obs_dict = {}
        obs_dict['time'] = np.array([sim.data.time])
        obs_dict['hand_qpos'] = sim.data.qpos[:].copy()
        obs_dict['hand_qvel'] = sim.data.qvel[:].copy() * self.dt
        if sim.model.na > 0:
            obs_dict['act'] = sim.data.act[:].copy()

        # reach error
        obs_dict['obj_pos'] = sim.data.site_xpos[self.object_sid]
        obs_dict['palm_pos'] = sim.data.site_xpos[self.palm_sid]
        obs_dict['reach_err'] = np.array(obs_dict['palm_pos']) - np.array(obs_dict['obj_pos'])
        return obs_dict

    def get_reward_dict(self, obs_dict):
        reach_dist = np.linalg.norm(obs_dict['reach_err'], axis=-1)
        act_mag = np.linalg.norm(self.obs_dict['act'], axis=-1) / self.sim.model.na if self.sim.model.na != 0 else 0
        far_th = self.far_th
        near_th = 0.1 
        drop = reach_dist > self.drop_th
        rwd_dict = collections.OrderedDict((
            # Optional Keys
            ('reach', -1. * reach_dist),
            ('bonus', 1. * (reach_dist < 2 * near_th) + 1. * (reach_dist < near_th)),
            ('act_reg', -1. * act_mag),
            ('penalty', -1. * (reach_dist > far_th)),
            # Must keys
            ('sparse', -1. * reach_dist),
            ('solved', reach_dist < near_th),
            ('done', reach_dist < near_th),
        ))
        rwd_dict['dense'] = np.sum([wt * rwd_dict[key] for key, wt in self.rwd_keys_wt.items()], axis=0)
        return rwd_dict

    # generate a valid target
    def generate_target_pose(self):
        random_index = np.random.randint(0, len(self.obj_xyz_range))
        self.sim.model.body_pos[self.object_bid] = self.obj_xyz_range[random_index]
        self.sim.forward()

    def reset(self, reset_qpos=None, reset_qvel=None):

        # randomize init arms pose
        if self.qpos_noise_range is not None:
            reset_qpos_local = self.init_qpos + self.qpos_noise_range*(self.sim.model.jnt_range[:,1]-self.sim.model.jnt_range[:,0])
            reset_qpos_local[-6:] = self.init_qpos[-6:]
        else:
            reset_qpos_local = reset_qpos

        self.generate_target_pose()
        self.robot.sync_sims(self.sim, self.sim_obsd)
        obs = super().reset()
        return obs